In [8]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from subprocess import Popen
from time import sleep
import pandas as pd
import sqlite3
import os

In [10]:
def start_google_chrome(url):

    # Chrome start by specifying port
    chrome_path = r'"C:\Program Files\Google\Chrome\Application\chrome.exe" -remote-debugging-port=9222 --user-data-dir="C:\temp"'
    Popen(chrome_path)
    sleep(1)
    # Chrome options
    options = ChromeOptions()
    options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    # Get chrome with specified port
    # driver_path = r"C:\python\chromedriver\chromedriver.exe"
    # driver = Chrome(executable_path=driver_path, options=options)

    new_driver = ChromeDriverManager().install()
    service = Service(executable_path=new_driver)
    driver = Chrome(service=service, options=options)

    # open url
    driver.get(url)
    driver.implicitly_wait(30)
    sleep(1)

    return driver

In [3]:
URL = f"https://daidata.goraggio.com/"
STORES = {"EXA FIRST" : 101262}
hall_id = STORES["EXA FIRST"]

url = URL + str(hall_id)
print(url)

https://daidata.goraggio.com/101262


In [7]:
driver = start_google_chrome(url)

In [5]:

store_name = driver.title.replace(" - 台データオンライン", "")
store_name = store_name.replace(" ", "_")
print(store_name)

EXA_FIRST


In [6]:
# 機種名入力
search_word = "ジャグラー"
input_box = driver.find_element(By.NAME, "machine_name")
if input_box:
    input_box.clear()
    sleep(0.5)
    input_box.send_keys(search_word, Keys.ENTER)

In [178]:
# 機種名取得してクリック
models = driver.find_elements(By.CLASS_NAME, "model_name")
model_names = [model.text for model in models]
model_name = model_names[0].replace(" ", "_")
models[0].click()

WebDriverWait(driver, 10).until(
    EC.invisibility_of_element_located((By.CLASS_NAME, "table.sorter"))
)

True

In [207]:
# 日付変更
DAYS_AGO = 7
select_elem = driver.find_element(By.NAME, "hist_num")
select = Select(select_elem)
dates = [option.text for option in select.options]

select.select_by_visible_text(dates[DAYS_AGO])

WebDriverWait(driver, 10).until(
    EC.invisibility_of_element_located((By.CLASS_NAME, "table.sorter"))
)

print(dates[DAYS_AGO])

2025-03-18


In [208]:
# ヘッダー（固定）

# データフレーム化・保存
file_name = f"../csv/{store_name}_{model_name}_{dates[DAYS_AGO]}.csv"
 
if not os.path.exists(file_name):
    # テーブル取得
    rows = driver.find_elements(By.CSS_SELECTOR, 'table tr')
    
    columns = rows[0].text.split()
    data = []
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, 'td')
        
        if cols:
            data.append([col.text.strip() for col in cols[1:]])
    
    print(f"{len(data)} 件のデータ取得完了")
    
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(file_name, index=False, encoding="utf-8-sig")
    print(f"データ保存完了： {file_name}")
else:
    print(f"データは既に保存されています： {file_name}")
    df = pd.read_csv(file_name)

81 件のデータ取得完了
データ保存完了： ../csv/EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_2025-03-18.csv


In [209]:
# driver.quit()


In [ ]:
# データベースに接続（ファイルがなければ作成される）
conn = sqlite3.connect("../db/pachislo.db")
cursor = conn.cursor()


conn.commit()

for row in conn.execute("PRAGMA table_info(slot_data)"):
    print(row)

NameError: name 'sqlite3' is not defined

In [211]:
df["date"] = dates[DAYS_AGO]
df["hall_id"] = hall_id
df["model"] = model_name

In [212]:
df_sqlite = df.rename(columns={
    "台番号": "unit_no", "累計スタート": "start", "BB回数": "bb", "RB回数": "rb",
    "ART回数": "art", "最大持玉": "max_medals", "BB確率": "bb_rate", "RB確率": "rb_rate",
    "ART確率": "art_rate", "合成確率": "total_rate", "前日最終スタート": "last_start"
})

df_sqlite.head()

,unit_no,start,bb,rb,art,max_medals,bb_rate,rb_rate,art_rate,total_rate,last_start,date,hall_id,model
0,1001,4420,13,14,0,1194,1/340.0,1/315.7,0.0,163.7,393,2025-03-18,101262,SﾏｲｼﾞｬｸﾞﾗｰⅤKD
1,1002,1751,5,4,0,416,1/350.2,1/437.8,0.0,194.6,356,2025-03-18,101262,SﾏｲｼﾞｬｸﾞﾗｰⅤKD
2,1003,4027,12,11,0,898,1/335.6,1/366.1,0.0,175.1,242,2025-03-18,101262,SﾏｲｼﾞｬｸﾞﾗｰⅤKD
3,1004,1501,5,3,0,402,1/300.2,1/500.3,0.0,187.6,276,2025-03-18,101262,SﾏｲｼﾞｬｸﾞﾗｰⅤKD
4,1005,7263,26,21,0,1066,1/279.3,1/345.9,0.0,154.5,54,2025-03-18,101262,SﾏｲｼﾞｬｸﾞﾗｰⅤKD


In [213]:
# SQLiteに書き込む
df_sqlite.to_sql("slot_data", conn, if_exists="append", index=False)
conn.commit()